In [ ]:
import pandas as pd
from common import *

In [ ]:
# Convert SNOBAL grid cell from meter to square feet
CELL_AREA = (50.0 * 3.28084)**2
# MM in one foot
MM_IN_FT = 304.8
# Convert to cubic feet
CELL_AREA_IN_MM = CELL_AREA / MM_IN_FT

In [ ]:
# Cubic feet to cubic meter
FT3_to_M3 = (MM_IN_FT/1000)**3

In [ ]:
wy2018 = dict(time=slice("2017-10-01", "2018-09-30"))
wy2019 = dict(time=slice("2018-10-01", "2019-09-30"))
wy2020 = dict(time=slice("2019-10-01", "2020-09-30"))
wy2021 = dict(time=slice("2020-10-01", "2021-09-30"))

In [ ]:
zones = cbrfc_zones()

# Compare iSnobal SWI to USGS discharge gauge at Almont 

## USGS 

Source: https://nwis.waterdata.usgs.gov/co/nwis/dv/?site_no=09112500&agency_cd=USGS&amp;referred_module=sw

In [ ]:
PD_OPTIONS = dict(
    comment='#',
    parse_dates=True,
    index_col=0,
    names=['Date', 'Discharge'],
    header=0,
    sep='\t',
    dtype={'Discharge': np.float64},
    usecols=[2,3],
)
data = pd.read_csv(
    DATA_DIR / 'USGS-SWI-Almont.csv', 
    **PD_OPTIONS
)

data.Discharge = data.Discharge * 60 * 60 * 24 * FT3_to_M3

Area average from iSnobal vs CSV

## CBRFC

In [ ]:
PD_OPTIONS = dict(
    comment='#',
    parse_dates=True,
    index_col=0,
    names=['Date', 'SWE'],
    header=0,
    sep=',',
    dtype={'Discharge': np.float64},
)
cbrfc_HLF = pd.read_csv(
    CBRFC_DIR / 'SWI/ALEC2HLF.RAIM.MA.csv', 
    **PD_OPTIONS
)
cbrfc_HMF = pd.read_csv(
    CBRFC_DIR / 'SWI/ALEC2HMF.RAIM.MA.csv', 
    **PD_OPTIONS
)
cbrfc_HUF = pd.read_csv(
    CBRFC_DIR / 'SWI/ALEC2HUF.RAIM.MA.csv', 
    **PD_OPTIONS
)
cbrfc = cbrfc_HLF + cbrfc_HMF + cbrfc_HUF
cbrfc = cbrfc["2017-10-01":"2021-09-30"]
cbrfc = (cbrfc.SWE * (INCH_TO_MM / 1000)) * (zones[zones != 0].size * 50**2)

## SNOBAL

In [ ]:
client = start_cluster(8, 24)
client_ip_and_port(client)

In [ ]:
snobal_em = xr.open_mfdataset(
    (SNOBAL_DIR / 'GMD/wy*/erw/*/em.nc').as_posix(),
    data_vars=['SWI'],
    parallel=True,
)
snobal_em.coords['mask'] = (('y', 'x'), zones)
snobal_em['time'] = snobal_em.time + np.timedelta64(2,'h')

In [ ]:
swi = (
    (snobal_em.where(snobal_em.mask != 0).SWI / 1000) * 50**2
).sum(['x', 'y']).compute()

In [ ]:
client.shutdown()

### Moving 7-day average SNOBAL vs. USGS

In [ ]:
figure = plt.figure(dpi=300, figsize=(10,3))
ax = plt.gca()

swi.rolling(time=7, center=True).mean().plot(ax=ax, label='HRRR-iSnobal SWI', color='peru', alpha=0.8, lw=1)
cbrfc.rolling(28, center=True).mean().plot(ax=ax, label='SNOW-17 SWI', color='olivedrab', alpha=0.8, lw=1.2, ls=(0, (5, 1)))
data.rolling(7, center=True).mean().Discharge.plot(ax=ax, label='Stream Discharge', color='cadetblue', alpha=0.8, lw=1)

locator = mdates.AutoDateLocator(maxticks=20)

ax.xaxis.set_major_locator(locator)
ax.xaxis.set_minor_locator(mdates.MonthLocator())

formatter = mdates.ConciseDateFormatter(locator)
ax.xaxis.set_major_formatter(formatter)

index = 0
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=0, horizontalalignment='center')
    if index in [1, 5, 9, 13]:
        label.set(fontstyle='italic', fontweight=600)
    index += 1

ax.set_xlim(left=data.index.min(), right=swi.time.max().values)
ax.set_ylim(bottom=-1000)
ax.ticklabel_format(axis='y', useMathText=True)

plt.xlabel('')
plt.ylabel(r'Mean Volume ($\frac{m^3}{7 \ days}$)')
plt.legend(loc='upper center', ncol=3, bbox_to_anchor=(0.55, 1.14), frameon=False);